In [1]:
using RigidBodyDynamics
using RigidBodyDynamics.TreeDataStructure
using RigidBodyDynamics.OdeIntegrators
using RigidBodyTreeInspector

adding: /Users/twan/code/julia/RigidBodyDynamics/v0.5/DrakeVisualizer/src/lcmtypes to the python path


In [2]:
const DRAKE_DISTRO = "$(ENV["HOME"])/code/drake-distro"
urdf = "$DRAKE_DISTRO/drake/examples/Valkyrie/urdf/urdf/valkyrie_A_sim_drake_one_neck_dof_wide_ankle_rom.urdf"
package_path = ["$DRAKE_DISTRO/drake/examples"]
mechanism = RigidBodyDynamics.parse_urdf(Float64, urdf);

In [3]:
vis = parse_urdf(urdf, mechanism; package_path=package_path);

In [4]:
# Reroot at the left foot
pelvis = vertex_data(findfirst(v -> name(vertex_data(v)) == "pelvis", tree(mechanism)))
leftFoot = vertex_data(findfirst(v -> name(vertex_data(v)) == "leftFoot", tree(mechanism)))
leftFootToWorld = Joint("", Fixed{Float64}())
world = root_body(mechanism)
flippedJointMapping = reattach!(mechanism, pelvis, 
    world, leftFootToWorld, Transform3D{Float64}(leftFootToWorld.frameBefore, world.frame), 
    leftFoot, Transform3D{Float64}(leftFoot.frame, leftFootToWorld.frameAfter))
remove_fixed_joints!(mechanism);

In [5]:
const passive_dynamics! = (vd::AbstractArray, t, state) -> begin
    dynamics!(result, state)
    copy!(vd, result.v̇)
    nothing
end

(::#18) (generic function with 1 method)

In [6]:
type DrakeVisualizerSink <: OdeResultsSink
    vis::DrakeVisualizer.Visualizer
    Δt::Float64
    lastUpdateTime::Float64
    
    DrakeVisualizerSink(vis::DrakeVisualizer.Visualizer, Δt::Float64) = new(vis, Δt, -Inf)
end

function OdeIntegrators.initialize(sink::DrakeVisualizerSink, t, state)
    DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
    sink.lastUpdateTime = -Inf
end

function OdeIntegrators.process(sink::DrakeVisualizerSink, t, state)
    if t > sink.lastUpdateTime + sink.Δt
        draw(sink.vis, state)
        sink.lastUpdateTime = t
    end
end

In [7]:
# Simulation
const state = MechanismState(Float64, mechanism)
const result = DynamicsResult(Float64, mechanism)
const integrator = MuntheKaasIntegrator(passive_dynamics!, runge_kutta_4(Float64), DrakeVisualizerSink(vis, 1e-2));

In [10]:
zero!(state)
integrate(integrator, state, 1., 1e-4)